In [391]:
import tabula
import pandas as pd
import re

In [473]:
# Read pdf into a list of DataFrame
dfs = tabula.read_pdf("2021_enchantments_lottery_stats.pdf", lattice=True, pages='all')

In [474]:
column_set = []
for df in dfs:
#     print(df.shape)
    if 'Unnamed: 0' in df.columns:
        df.columns = df.iloc[0]
#         print('replaced')
    if list(df.columns) not in column_set:
        column_set.append(list(df.columns))

In [475]:
len(column_set[0]) # only continue if column_set is just one set of the right columns

17

In [595]:
merged = pd.concat(dfs)

In [596]:
merged.shape

(58466, 17)

In [597]:
merged.columns

Index(['Preferred Entry\rDate 1', 'Preferred Division 1',
       'Minimum\rAcceptable\rGroup Size 1', 'Maximum\rRequested\rGroup Size 1',
       'Preferred Entry Date 2', 'Preferred Division 2',
       'Minimum\rAcceptable\rGroup Size 2', 'Maximum\rRequested Group\rSize 2',
       'Preferred Entry Date\r3', 'Preferred Division 3',
       'Minimum\rAcceptable\rGroup Size 3', 'Maximum\rRequested\rGroup Size 3',
       'Results Status', 'Awarded\rPreference', 'Awarded Entry\rDate',
       'Awarded\rEntrance\rCode/Name', 'Awarded\rGroup Size'],
      dtype='object', name=0)

In [598]:
merged.iloc[0] == merged.columns

0
Preferred Entry\rDate 1              True
Preferred Division 1                 True
Minimum\rAcceptable\rGroup Size 1    True
Maximum\rRequested\rGroup Size 1     True
Preferred Entry Date 2               True
Preferred Division 2                 True
Minimum\rAcceptable\rGroup Size 2    True
Maximum\rRequested Group\rSize 2     True
Preferred Entry Date\r3              True
Preferred Division 3                 True
Minimum\rAcceptable\rGroup Size 3    True
Maximum\rRequested\rGroup Size 3     True
Results Status                       True
Awarded\rPreference                  True
Awarded Entry\rDate                  True
Awarded\rEntrance\rCode/Name         True
Awarded\rGroup Size                  True
Name: 0, dtype: bool

In [599]:
df = merged.mask((merged == merged.columns)).dropna(how='all')

In [600]:
merged.shape[0] - df.shape[0]

514

In [601]:
df.columns = [r.replace('\r',' ') for r in list(df.columns)]

In [602]:
list(df.columns)

['Preferred Entry Date 1',
 'Preferred Division 1',
 'Minimum Acceptable Group Size 1',
 'Maximum Requested Group Size 1',
 'Preferred Entry Date 2',
 'Preferred Division 2',
 'Minimum Acceptable Group Size 2',
 'Maximum Requested Group Size 2',
 'Preferred Entry Date 3',
 'Preferred Division 3',
 'Minimum Acceptable Group Size 3',
 'Maximum Requested Group Size 3',
 'Results Status',
 'Awarded Preference',
 'Awarded Entry Date',
 'Awarded Entrance Code/Name',
 'Awarded Group Size']

In [603]:
df = df.reindex()

In [604]:
def clean_numeric(x):
    try:
        x = int(x)
#         print(x)
    except:
#         print("nope")
        pass
    finally:
        x = str(x)
    new_x = filter(str.isdigit, x)
    new_x = "".join(new_x)
    return new_x

In [605]:
def clean_zone_name(x):
    replacement_awarded_entrance = {'Enchantment Z': 'Core Enchantment Zone',
                    'tmile/Caroline Z': 'Eightmile/Caroline Zone',
                    'e/Caroline Zone': 'Eightmile/Caroline Zone (stock)',
                    'uart Zone (stoc': 'Stuart Zone (stock)',
                    'Eightmile/Caroline Zone (s': 'Eightmile/Caroline Zone (stock)',
                    'Eightmile/Caroline Zone (st': 'Eightmile/Caroline Zone (stock)'}
    try:
        new_x = replacement_awarded_entrance[x]
    except:
        new_x = x
    return new_x

In [606]:
def clean_date(x):
    try:
        new_x = re.sub('[^0-9/]+', '', x)
    except:
        new_x = x
    return new_x

In [607]:
for col in df.columns:
    print(col)
    print()
    if 'Group Size' in col or 'Awarded Preference' in col:
        print('cleaning numeric!')
        df[col] = df[col].apply(clean_numeric)
    if 'Division' in col or 'Entrance' in col:
        print('cleaning zone names!')
        df[col] = df[col].apply(clean_zone_name)
    if 'Date' in col:
        print('cleaning dates!')
        df[col] = df[col].apply(clean_date)
    print(df[col].value_counts())
    print()

Preferred Entry Date 1

cleaning dates!
8/6/21      1211
8/5/21      1084
8/13/21     1026
7/22/21     1018
8/12/21      936
            ... 
10/26/21       2
10/24/21       2
10/19/21       1
10/21/21       1
10/27/21       1
Name: Preferred Entry Date 1, Length: 167, dtype: int64

Preferred Division 1

cleaning zone names!
Core Enchantment Zone              42530
Colchuck Zone                       7249
Snow Zone                           4480
Stuart  Zone                        2786
Eightmile/Caroline Zone              804
Eightmile/Caroline Zone (stock)       52
Stuart Zone (stock)                   51
Name: Preferred Division 1, dtype: int64

Minimum Acceptable Group Size 1

cleaning numeric!
4    16131
8    13554
6    10352
2     9317
5     4126
3     3253
7      702
1      517
Name: Minimum Acceptable Group Size 1, dtype: int64

Maximum Requested Group Size 1

cleaning numeric!
4    16131
8    13554
6    10352
2     9317
5     4126
3     3253
7      702
1      517
Name: Maximum 

In [608]:
df.to_csv('2021_enchantments_lottery_stats.csv')

In [609]:
df.columns

Index(['Preferred Entry Date 1', 'Preferred Division 1',
       'Minimum Acceptable Group Size 1', 'Maximum Requested Group Size 1',
       'Preferred Entry Date 2', 'Preferred Division 2',
       'Minimum Acceptable Group Size 2', 'Maximum Requested Group Size 2',
       'Preferred Entry Date 3', 'Preferred Division 3',
       'Minimum Acceptable Group Size 3', 'Maximum Requested Group Size 3',
       'Results Status', 'Awarded Preference', 'Awarded Entry Date',
       'Awarded Entrance Code/Name', 'Awarded Group Size'],
      dtype='object')

In [612]:
df = df.reset_index()

In [613]:
# minimum and maximum group size is the same...?!
first_choice = ['Preferred Entry Date 1', 'Preferred Division 1',
       'Minimum Acceptable Group Size 1']
second_choice = ['Preferred Entry Date 2', 'Preferred Division 2',
       'Minimum Acceptable Group Size 2']
third_choice = ['Preferred Entry Date 3', 'Preferred Division 3',
       'Minimum Acceptable Group Size 3']

In [614]:
preferences = [df[first_choice], df[second_choice], df[third_choice]]

In [615]:
for d in preferences:
    d['Result'] = ""

<ipython-input-615-e87a676c40f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Result'] = ""


In [616]:
preferences[0]

,Preferred Entry Date 1,Preferred Division 1,Minimum Acceptable Group Size 1,Result
0,6/21/21,Colchuck Zone,2,
1,8/31/21,Colchuck Zone,2,
2,8/19/21,Colchuck Zone,4,
3,8/12/21,Colchuck Zone,8,
4,7/9/21,Colchuck Zone,4,
...,...,...,...,...
57947,9/8/21,Stuart Zone (stock),6,
57948,9/27/21,Stuart Zone (stock),2,
57949,9/30/21,Stuart Zone (stock),4,
57950,10/7/21,Stuart Zone (stock),4,


In [617]:
for index, preference in enumerate(df['Awarded Preference']):
    if preference != '':
#         print('index: ', index)
        awarded_df = preferences[int(preference)-1]
        awarded_df.at[index, 'Result'] = 1
#         awarded_df.at[index, 'Results Status'] = df.iloc[index]['Results Status']

In [618]:
preferences[1].head(68)

,Preferred Entry Date 2,Preferred Division 2,Minimum Acceptable Group Size 2,Result
0,6/28/21,Colchuck Zone,2,
1,9/1/21,Colchuck Zone,2,
2,8/19/21,Core Enchantment Zone,4,
3,8/19/21,Colchuck Zone,8,
4,7/9/21,Snow Zone,4,
...,...,...,...,...
63,8/26/21,Colchuck Zone,2,
64,9/24/21,Colchuck Zone,4,
65,7/3/21,Core Enchantment Zone,4,
66,7/19/21,Core Enchantment Zone,4,


In [619]:
for d in preferences:
    d.columns = ['Preferred Entry Date', 'Preferred Zone', 'Group Size', 'Result']

In [646]:
clean_df = pd.concat(preferences)

In [648]:
clean_df['Preferred Entry Date'] = pd.to_datetime(clean_df['Preferred Entry Date'])

In [657]:
clean_df['Day of Week'] = clean_df['Preferred Entry Date'].dt.day_name()

In [663]:
clean_df['Month'] = clean_df['Preferred Entry Date'].dt.month_name()

In [665]:
clean_df['Day of Year'] = clean_df['Preferred Entry Date'].dt.dayofyear

In [683]:
clean_df.shape

(173856, 7)

In [684]:
# sometimes people don't put in a 2nd or 3rd preference
clean_df.dropna(inplace=True)

In [685]:
clean_df.shape

(171560, 7)

In [686]:
clean_df.to_csv('modeling_enchantments_data.csv', index=False)